# Download weight

In [ ]:
!git clone https://github.com/BinhPQ2/Han_Nom_OCR.git
!git clone https://github.com/THU-MIG/yolov10.git

In [1]:
!gdown --id 11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w -O file.zip
!unzip -q file.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1HDmwi6gvaSZk3opgnXKXBoIrFHgKGlq4
From (redirected): https://drive.google.com/uc?id=1HDmwi6gvaSZk3opgnXKXBoIrFHgKGlq4&confirm=t&uuid=4ce6b6ea-c4cd-4dde-9539-217af9c175ff
To: /content/file.zip
100% 271M/271M [00:11<00:00, 24.3MB/s]


# Detection

In [17]:
!pip install -q -r requirements.txt
!pip install -q -e ./yolov10/
# !pip install supervision -q # already included in the requirement.txt
# !pip install ultralytics -q

Obtaining file:///content/yolov10
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.1.34-0.editable-py3-none-any.whl size=20539 sha256=f227f7e61e46526bbe1fc057ba3baee5a3d942d930ec268af4c9ba72e462cba4
  Stored in directory: /tmp/pip-ephem-wheel-cache-oy5m58v0/wheels/51/93/e8/22d2e815ced343915c15d86b2a00d95eb0a997d012527fbea7
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.1.34
    Uninstalling ultralytics-8.1.34:
      Successfully uninstalled ultralytics-8.1.34


In [1]:
!pip show ultralytics

Name: ultralytics
Version: 8.1.34
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /usr/local/lib/python3.10/dist-packages
Editable project location: /content/yolov10
Requires: matplotlib, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 


In [2]:
import os
import random
import numpy as np
from tqdm import tqdm
import pickle
import cv2
import supervision as sv
import matplotlib.pyplot as plt
from ultralytics import YOLOv10

In [5]:
def save_crop(result, save_path):
    os.makedirs(save_path, exist_ok=True)

    conf_s = result.boxes.conf.detach().cpu().numpy()
    xyxy_s = result.boxes.xyxy.detach().cpu().numpy()
    idx_s = [i for i in range(len(conf_s))]
    img_name = os.path.basename(result.path)
    img = result.orig_img
    for idx, conf, xyxy in zip(idx_s, conf_s, xyxy_s):
        # name = f"oder_{idx}__conf_{conf}__name_{img_name}"
        name = img_name

        height=result.orig_img.shape[0]
        width=result.orig_img.shape[1]

        height_extend_ratio = 0.05
        width_extend_ratio = 0.1

        x0, y0, x1, y1 = xyxy.tolist()
        x0 = int(max(0, x0 - width_extend_ratio*abs(x1 - x0)))
        x1 = int(min(width, x1 + width_extend_ratio*abs(x1 - x0)))
        y0 = int(max(0, y0 - height_extend_ratio*abs(y1 - y0)))
        y1 = int(min(height, y1 + height_extend_ratio*abs(y1 - y0)))
        crop_image = img[y0:y1, x0:x1]

        cv2.imwrite(os.path.join(save_path, name), crop_image)

In [6]:
input_image_folder = "./input_image"
save_path = "./output_detection"
model = YOLOv10('./weight/detection_yolov10.pt')

results = model.predict(source=input_image_folder, conf=0.5, stream=True, save_txt=True)
for idx, result in enumerate(results):
    save_crop(result, save_path)


image 1/23 /content/input_image/02b02efc5d6b4579a0cae31e1e362555.jpg: 640x480 1 couplet, 1038.8ms
Processed 02b02efc5d6b4579a0cae31e1e362555.jpg
659 74 804 1986
image 2/23 /content/input_image/0ff04276ca994b12a50f51f88a9373fc.jpg: 640x352 1 couplet, 782.0ms
Processed 0ff04276ca994b12a50f51f88a9373fc.jpg
232 0 322 883
image 3/23 /content/input_image/1a2291615b4e4c288a152b4886ac300e.jpg: 640x480 1 couplet, 1019.9ms
Processed 1a2291615b4e4c288a152b4886ac300e.jpg
364 37 427 904
image 4/23 /content/input_image/1a5634d0281b4b1aac6e616826c30d4b.jpg: 640x320 2 couplets, 737.9ms
Processed 1a5634d0281b4b1aac6e616826c30d4b.jpg
167 54 307 1227
Processed 1a5634d0281b4b1aac6e616826c30d4b.jpg
393 66 508 1227
image 5/23 /content/input_image/1a7296dbb22748e49b656df5cc6743b5.jpg: 640x288 2 couplets, 672.1ms
Processed 1a7296dbb22748e49b656df5cc6743b5.jpg
427 341 558 1764
Processed 1a7296dbb22748e49b656df5cc6743b5.jpg
200 626 318 1286
image 6/23 /content/input_image/1b094caec3984b8eb1c929b60a8d2b08.jpg: 

# Recognition

In [7]:
!pip install -q paddleocr --upgrade
!pip install -q paddlepaddle-gpu --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 1.6 MB/s eta 0:00:00


In [8]:
!python3 recognition/predict_rec.py --image_dir="./output_detection" --rec_model_dir='./weight/recognition_PPOCRLABEL'

/usr/local/lib/python3.10/dist-packages/paddle/base/framework.py:688: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(
Traceback (most recent call last):
  File "/content/recognition/predict_rec.py", line 32, in <module>
    import tools.infer.utility as utility
ModuleNotFoundError: No module named 'tools.infer'
